<a href="https://colab.research.google.com/github/choprahetarth/RFP-AI/blob/main/RFP_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer
!pip install PyPDF2
!pip install pdfPlumber

In [ ]:
''' implementing the tutorial at https://towardsdatascience.com/creating-a-custom-classifier-for-text-cleaning-a2a1fc818935'''
from collections import Counter
from IPython.display import clear_output
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from textblob import TextBlob
import io
import math
import numpy as np
import pickle
import pandas as pd
import string
import re
from PyPDF2 import PdfFileReader, PdfFileWriter
import os
import os

In [ ]:
#INPUT AREA #
print("Welcome to AutoCBC Generator")
project_name = input("please let me know the name of the project - >")
start_of_the_section = input("Please Let Me Know the start of your section (do a -1 from the actual start) - >")
end_of_section  = input("Please Let Me Know the end of your section  - >")
number_of_section = input("Please tell the section number - >") 

Welcome to AutoCBC Generator
please let me know the name of the project - >CMRL PHASE II
Please Let Me Know the start of your section (do a -1 from the actual start) - >300
Please Let Me Know the end of your section  - >318
Please tell the section number - >13


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/RFPAI/Part_2_ERTS.pdf'
pdf = PdfFileReader(path, "rb")

pdf_writer = PdfFileWriter()

#for page in range(283, 299):
#    pdf_writer.addPage(pdf.getPage(page))

for page in range(int(start_of_the_section), int(end_of_section)):
    pdf_writer.addPage(pdf.getPage(page))

output_fname = "/content/drive/MyDrive/Colab Notebooks/RFPAI/Output.pdf"

with open(output_fname, 'wb') as out:
    pdf_writer.write(out)

print ("PDF file has been split")

PDF file has been split


In [ ]:
def read_pdf(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    #laparams = LAParams()
    #device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    device = TextConverter(rsrcmgr, retstr)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True): 
        interpreter.process_page(page)
    text = retstr.getvalue()
    text = " ".join(text.replace(u"\xa0", " ").strip().split())  
    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/RFPAI')

In [ ]:
a = read_pdf('Output.pdf')

In [ ]:
pickle_out = open("serializedPDF.pickle","wb")
pickle.dump(a, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("serializedPDF.pickle","rb")
a = pickle.load(pickle_in)

In [ ]:
a

'Chennai Metro Rail Project Phase – II Bid No. ARE03 Part 2 – Section VI ERTS – Communications CMRL / Rev. B 13-1 December 2020 13. COMMUNICATIONS 13.1 CITED REFERENCES EIA-SE-101 Amplifiers for Sound Equipment EIA-SE-103 Speakers for Sound Equipment EIA-SE-105 Microphones for Sound Equipment IEC 60571 Hardware System IEC 61375-1 Train Communications Network – Part 1: Technical Specification MIL-STD-1472 Human Engineering Design Criteria RGS GM/RT/2472 Data Recorders on Trains – Design Requirements BS EN 60529 Specification for Degrees of Protection Provided by Enclosures Note: Any other equivalent standards to the above standards may be followed subject to approval of CMRL. All the standards used shall be latest issued revisions. 13.2 GENERAL 13.2.1 The Contractor shall conform to applicable international standards and best practices for communications requirements and shall be responsible for design and interface coordination, installing and testing the entire train communications sy

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
def pdf_to_df(path):
    content = a
    blob = TextBlob(content)
    words = blob.words
    df = pd.DataFrame({'sentence': words, 'label': np.nan})
    df['sentence'] = df.sentence.apply(''.join)
    return df
df = pdf_to_df('Output.pdf')
df.head()

,sentence,label
0,Chennai,NaN
1,Metro,NaN
2,Rail,NaN
3,Project,NaN
4,Phase,NaN


In [ ]:
df.to_pickle('brakes.pickle')

In [ ]:
column_names = ['requirementID', 'requirement', 'Label']
dfCBC = pd.DataFrame(columns= column_names)
dfCBC

,requirementID,requirement,Label


In [ ]:
len(re.findall(r"("+str(number_of_section)+"(?!-)(\.\d+)*)",df['sentence'][1]))

0

In [ ]:
"(^"+str(number_of_section)+"(?!-)(\.\d+)*(?!\d+))"

'(^13(?!-)(\\.\\d+)*(?!\\d+))'

In [ ]:
def selectSentence(year):
  #registery = re.findall(r"[-+]?\d*\.\d+|\d+", year)
  #registery = re.findall(r"\d*\.\d+|\d+", year)
  #registery = re.findall(r"\b12\.\d+|\b12\b",year)
  #registery = re.findall(r"(12(?!-)(\.\d+)*)",year)
  #registery  = re.findall(r"(12(?!-)(\.\d+)*(?!\d+))",year)
  #registery  = re.findall(r"(^12(?!-)(\.\d+)*(?!\d+))",year)
  registery = re.findall(r"(^"+str(number_of_section)+"(?!-)(\.\d+)*(?!\d+))",year)
  if len(registery) == 1:
    registery = registery[0][0]
  # initialize an empty string
  str1 = ""   
  # traverse in the string  
  for ele in registery: 
    str1 += ele  
    
  return str1 

df['sentence2'] = df['sentence'].apply(lambda x: selectSentence(x))
cut_df = df[df.astype(str)['sentence2'] != '']
number_indices = cut_df.index.to_list()
cut_df = cut_df.drop(columns=['sentence2'])
cut_df.to_excel("12 extracts.xlsx")

In [ ]:
sentence1 = [df.sentence[0], df.sentence[1],df.sentence[3]]
(' ').join(sentence1)

'Chennai Metro Project'

In [ ]:
sentence1 = []
for k in range(3):
  print(k)
  sentence1.append(df.sentence[k])
(' ').join(sentence1)

0
1
2


'Chennai Metro Rail'

In [ ]:
sentence2 = []
for i in range(len(number_indices)-1):
  difference = number_indices[i+1] - number_indices[i]
  sentence1 = []
  for k in range(difference):
    sentence1.append(df.sentence[k+number_indices[i]])
  p = (' ').join(sentence1)
  sentence2.append(p)
    #print(k)

In [ ]:
sentence2

['13 COMMUNICATIONS',
 '13.1 CITED REFERENCES EIA-SE-101 Amplifiers for Sound Equipment EIA-SE-103 Speakers for Sound Equipment EIA-SE-105 Microphones for Sound Equipment IEC 60571 Hardware System IEC 61375-1 Train Communications Network – Part 1 Technical Specification MIL-STD-1472 Human Engineering Design Criteria RGS GM/RT/2472 Data Recorders on Trains – Design Requirements BS EN 60529 Specification for Degrees of Protection Provided by Enclosures Note Any other equivalent standards to the above standards may be followed subject to approval of CMRL All the standards used shall be latest issued revisions',
 '13.2 GENERAL',
 '13.2.1 The Contractor shall conform to applicable international standards and best practices for communications requirements and shall be responsible for design and interface coordination installing and testing the entire train communications system',
 '13.2.2 The system components shall be modular and shall be removable for servicing without affecting other func

In [ ]:
sentence3 = []
sentence2[9].split()[0]
#sentence2[9].replace((sentence2[9].split()[0])+" ",'')

'13.2.7'

In [ ]:
for i in range(len(sentence2)):
  dfCBC.loc[i] = [sentence2[i].split()[0]] + [np.nan] + [sentence2[i].replace((sentence2[i].split()[0])+" ",'')]

In [ ]:
dfCBC.to_excel('CBC.xlsx')

In [ ]:
'''def manually_label(pickle_file):
    print('Is this requirement related to brakes? Type 1 if yes. \n')
    df = pd.read_pickle(pickle_file)
    for index, row in df.iterrows():
        if pd.isnull(row.label):
            print(row.sentence)
            label = input()
            if label == '1':
                df.loc[index, 'label'] = 1
            if label == '':
                df.loc[index, 'label'] = 0
            clear_output()
            df.to_pickle('brakes.pickle')
            
    print('No more labels to classify!')

manually_label('brakes.pickle')'''

"def manually_label(pickle_file):\n    print('Is this requirement related to brakes? Type 1 if yes. \n')\n    df = pd.read_pickle(pickle_file)\n    for index, row in df.iterrows():\n        if pd.isnull(row.label):\n            print(row.sentence)\n            label = input()\n            if label == '1':\n                df.loc[index, 'label'] = 1\n            if label == '':\n                df.loc[index, 'label'] = 0\n            clear_output()\n            df.to_pickle('brakes.pickle')\n            \n    print('No more labels to classify!')\n\nmanually_label('brakes.pickle')"

In [ ]:
df.to_excel('extarcted text.xlsx')